In [1]:
import rtde_control
import rtde_receive
import numpy as np
import time
import threading
import sys

rtde_c = rtde_control.RTDEControlInterface("192.168.20.35")
rtde_r = rtde_receive.RTDEReceiveInterface("192.168.20.35")

home_pose = [-0.129, 0.3, 0.252, 3.151, 0, 0]
target = [-0.129, 0.3, 0.4, 3.151, 0, 0]

rtde_c.moveJ_IK(home_pose, 0.5, 0.3)

def update_z_axis():
    global force_data
    while True:
        force_data = rtde_r.getActualTCPForce()
        # print("TCP force: ", force_data[0:3])
        time.sleep(0.1)

# Start the thread for updating z_axis
# z_axis_thread = threading.Thread(target=update_z_axis, daemon=True)
# z_axis_thread.start()

z_axis = 0.2085

array_to_ur = np.loadtxt('C:\\git\\FRA_503-IOT\\Nodejs\\Processed_image\\'+'pipo'+'CV'+'ur_array.txt')

step = 1

for i in range(array_to_ur.shape[0]):
    if step != array_to_ur[:,2][i]:  
        #ยกปลายแขนขึ้นเพื่อวาดจุดต่อไป
        target_pose = [array_to_ur[:,0][i-1], array_to_ur[:,1][i-1], z_axis + 0.03 , 3.151, 0, 0]
        print('step: ', array_to_ur[:,2][i] , target_pose)
        rtde_c.moveJ_IK(target_pose, 1.2, 0.5)

        #เคลื่อนไปยังจุดต่อไป
        target_pose = [array_to_ur[:,0][i], array_to_ur[:,1][i], z_axis + 0.03 , 3.151, 0, 0]
        print('step: ', array_to_ur[:,2][i] , target_pose)
        rtde_c.moveJ_IK(target_pose, 1.2, 0.5)
        print('before contact paper: ', rtde_r.getActualTCPForce()[2])
        before_contact = rtde_r.getActualTCPForce()[2]

        #เอาแขนลง
        target_pose = [array_to_ur[:,0][i], array_to_ur[:,1][i], z_axis, 3.151, 0, 0]
        print('step: ', array_to_ur[:,2][i] , target_pose)
        rtde_c.moveJ_IK(target_pose, 1.2, 0.5)
        print('after contact paper: ', rtde_r.getActualTCPForce()[2])
        after_contact = rtde_r.getActualTCPForce()[2]
        print('Different_force', after_contact - before_contact)

        if after_contact - before_contact < 0.3:
            target_pose = [array_to_ur[:,0][i], array_to_ur[:,1][i], z_axis - 0.0005, 3.151, 0, 0]
            rtde_c.moveJ_IK(target_pose, 1.2, 0.5)
        elif after_contact - before_contact > 0.7:
            target_pose = [array_to_ur[:,0][i], array_to_ur[:,1][i], z_axis + 0.0005, 3.151, 0, 0]
            rtde_c.moveJ_IK(target_pose, 1.2, 0.5)

        step = array_to_ur[:,2][i]
    else: #ให้ปลายแขนเคลื่อนที่ไปตามจุด target point เรื่อยๆ
        target_pose = [array_to_ur[:,0][i], array_to_ur[:,1][i], z_axis, 3.151, 0, 0]
        # print(target_pose)
        rtde_c.moveJ_IK(target_pose, 1.2, 0.5)

rtde_c.moveJ_IK(home_pose, 0.5, 0.3)
rtde_c.disconnect()
rtde_r.disconnect()

step:  2.0 [-0.002, 0.376, 0.2385, 3.151, 0, 0]
step:  2.0 [-0.002, 0.34, 0.2385, 3.151, 0, 0]
before contact paper:  38.32760112840863
step:  2.0 [-0.002, 0.34, 0.2085, 3.151, 0, 0]
after contact paper:  38.58326608551128
Different_force 0.2471101373903224
step:  3.0 [-0.002, 0.341, 0.2385, 3.151, 0, 0]
step:  3.0 [-0.002, 0.281, 0.2385, 3.151, 0, 0]
before contact paper:  38.13378233918438
step:  3.0 [-0.002, 0.281, 0.2085, 3.151, 0, 0]
after contact paper:  38.56670296490345
Different_force 0.43292062571907053
step:  4.0 [-0.002, 0.281, 0.2385, 3.151, 0, 0]
step:  4.0 [-0.003, 0.34900000000000003, 0.2385, 3.151, 0, 0]
before contact paper:  38.162846735968664
step:  4.0 [-0.003, 0.34900000000000003, 0.2085, 3.151, 0, 0]
after contact paper:  38.19935730406489
Different_force 0.03651056809622588
step:  5.0 [-0.002, 0.34900000000000003, 0.2385, 3.151, 0, 0]
step:  5.0 [-0.003, 0.257, 0.2385, 3.151, 0, 0]
before contact paper:  37.94980135659735
step:  5.0 [-0.003, 0.257, 0.2085, 3.151